# Cleaning Global Superstore Dataset for Streamlit BI Dashboard

## Why Data Cleaning?

- ### Cleaning is the foundation of any professional data science or business intelligence project.
- ### Dirty, inconsistent, or incomplete data leads to misleading KPIs, inaccurate insights, and poor decision-making.
- ### For this project, I will clean the Global Superstore dataset to prepare it for building a high-quality Streamlit dashboard.
- ### The goal is to ensure the data is reliable, readable, and rich with features that can be visualized effectively.

### -----------------------------------------------------
### Step-by-Step Cleaning and Preparation
### -----------------------------------------------------

In [9]:
# Step 1: Import necessary libraries
import pandas as pd
import numpy as np

# Step 2: Load the dataset with correct encoding
df_raw = pd.read_csv("Global_Superstore2.csv", encoding='ISO-8859-1')

# Step 3: View original shape and first few rows
print("Original shape:", df_raw.shape)
df_raw.head()

Original shape: (51290, 24)


,Row ID,Order ID,Order Date,Ship Date,Ship Mode,Customer ID,Customer Name,Segment,City,State,...,Product ID,Category,Sub-Category,Product Name,Sales,Quantity,Discount,Profit,Shipping Cost,Order Priority
0,32298,CA-2012-124891,31-07-2012,31-07-2012,Same Day,RH-19495,Rick Hansen,Consumer,New York City,New York,...,TEC-AC-10003033,Technology,Accessories,Plantronics CS510 - Over-the-Head monaural Wir...,2309.650,7,0.0,762.1845,933.57,Critical
1,26341,IN-2013-77878,05-02-2013,07-02-2013,Second Class,JR-16210,Justin Ritter,Corporate,Wollongong,New South Wales,...,FUR-CH-10003950,Furniture,Chairs,"Novimex Executive Leather Armchair, Black",3709.395,9,0.1,-288.7650,923.63,Critical
2,25330,IN-2013-71249,17-10-2013,18-10-2013,First Class,CR-12730,Craig Reiter,Consumer,Brisbane,Queensland,...,TEC-PH-10004664,Technology,Phones,"Nokia Smart Phone, with Caller ID",5175.171,9,0.1,919.9710,915.49,Medium
3,13524,ES-2013-1579342,28-01-2013,30-01-2013,First Class,KM-16375,Katherine Murray,Home Office,Berlin,Berlin,...,TEC-PH-10004583,Technology,Phones,"Motorola Smart Phone, Cordless",2892.510,5,0.1,-96.5400,910.16,Medium
4,47221,SG-2013-4320,05-11-2013,06-11-2013,Same Day,RH-9495,Rick Hansen,Consumer,Dakar,Dakar,...,TEC-SHA-10000501,Technology,Copiers,"Sharp Wireless Fax, High-Speed",2832.960,8,0.0,311.5200,903.04,Critical


In [11]:
# Step 4: Drop unnecessary columns
columns_to_keep = [
    "Order Date", "Region", "Category", "Sub-Category",
    "Customer Name", "Customer ID",
    "Sales", "Profit", "Quantity", "Discount",
    "State", "City", "Segment"
]

df = df_raw[columns_to_keep].copy()
print("Retained columns only")

# Step 5: Handle duplicates
before_dupes = df.shape[0]
df.drop_duplicates(inplace=True)
after_dupes = df.shape[0]
print(f"Removed {before_dupes - after_dupes} duplicate rows")

Retained columns only
Removed 9 duplicate rows


In [13]:
# Step 6: Handle missing values
missing = df.isnull().sum()
print("Missing values per column:\n", missing)

# Drop rows with missing sales/profit; fill location infoo
df.dropna(subset=["Sales", "Profit"], inplace=True)
df.fillna({"State": "Unknown", "City": "Unknown"}, inplace=True)

Missing values per column:
 Order Date       0
Region           0
Category         0
Sub-Category     0
Customer Name    0
Customer ID      0
Sales            0
Profit           0
Quantity         0
Discount         0
State            0
City             0
Segment          0
dtype: int64


In [17]:
# Step 7: Convert Order Date to datetime
df["Order Date"] = pd.to_datetime(df["Order Date"], dayfirst=True,  errors="coerce")

# Drop rows where date conversion failed
before_datefix = df.shape[0]
df = df.dropna(subset=["Order Date"])
after_datefix = df.shape[0]
print(f"Dropped {before_datefix - after_datefix} rows with invalid dates")

Dropped 0 rows with invalid dates


In [19]:
# Step 8: Convert Sales and Profit to float
df["Sales"] = df["Sales"].astype(float)
df["Profit"] = df["Profit"].astype(float)
df["Quantity"] = df["Quantity"].astype(int)
df["Discount"] = df["Discount"].astype(float)

print("Converted numerical columns to appropriate types")

Converted numerical columns to appropriate types


In [21]:
# Step 9: Extract time-based features
df["Year"] = df["Order Date"].dt.year
df["Month"] = df["Order Date"].dt.strftime('%B')
df["Month_Num"] = df["Order Date"].dt.month
df["Weekday"] = df["Order Date"].dt.day_name()
df["Week_Num"] = df["Order Date"].dt.isocalendar().week

print("Extracted Year, Month, Weekday, etc.")

Extracted Year, Month, Weekday, etc.


In [23]:
# Step 10: Create meaningful derived features
df["Sales_per_Quantity"] = df["Sales"] / df["Quantity"]
df["Profit_per_Sale"] = df["Profit"] / df["Sales"]
df["Is_Profitable"] = np.where(df["Profit"] > 0, "Yes", "No")

print("Created new analytical features")

Created new analytical features


In [25]:
# Step 11: Confirm final shape and preview cleaned data
print("Final cleaned shape:", df.shape)
df.head()

Final cleaned shape: (51281, 21)


,Order Date,Region,Category,Sub-Category,Customer Name,Customer ID,Sales,Profit,Quantity,Discount,...,City,Segment,Year,Month,Month_Num,Weekday,Week_Num,Sales_per_Quantity,Profit_per_Sale,Is_Profitable
0,2012-07-31,East,Technology,Accessories,Rick Hansen,RH-19495,2309.650,762.1845,7,0.0,...,New York City,Consumer,2012,July,7,Tuesday,31,329.950,0.330000,Yes
1,2013-02-05,Oceania,Furniture,Chairs,Justin Ritter,JR-16210,3709.395,-288.7650,9,0.1,...,Wollongong,Corporate,2013,February,2,Tuesday,6,412.155,-0.077847,No
2,2013-10-17,Oceania,Technology,Phones,Craig Reiter,CR-12730,5175.171,919.9710,9,0.1,...,Brisbane,Consumer,2013,October,10,Thursday,42,575.019,0.177766,Yes
3,2013-01-28,Central,Technology,Phones,Katherine Murray,KM-16375,2892.510,-96.5400,5,0.1,...,Berlin,Home Office,2013,January,1,Monday,5,578.502,-0.033376,No
4,2013-11-05,Africa,Technology,Copiers,Rick Hansen,RH-9495,2832.960,311.5200,8,0.0,...,Dakar,Consumer,2013,November,11,Tuesday,45,354.120,0.109963,Yes


In [27]:
# Step 12: Save the cleaned dataset
df.to_csv("cleaned_global_superstore.csv", index=False)
print("Cleaned file saved as 'cleaned_global_superstore.csv'")

Cleaned file saved as 'cleaned_global_superstore.csv'


# Summary of Cleaning Steps Performed

- Loaded dataset using appropriate encoding (ISO-8859-1)
- Dropped irrelevant columns
- Removed duplicates
- Handled missing values smartly
- Parsed 'Order Date' into datetime format
- Converted main columns (Sales, Profit, etc.) to float/int
- Extracted new time-based features (Month, Year, Weekday)
- Created additional features:
  - Sales_per_Quantity
  - Profit_per_Sale
  - Is_Profitable
- Verified final shape and saved as a clean CSV for dashboard use

Now the dataset is ready for interactive BI dashboard development in Streamlit.